<a href="https://colab.research.google.com/github/atoye1/colab_notebook/blob/main/plotly_btc_traffic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import plotly.graph_objs as go
import pandas as pd

In [2]:
import numpy as np
import io
# 컬럼을 녹이기 위해 작은 데이터프레임을 추출하고, 이를 대상으로 테스트 해본다.

# csv_io = io.StringIO(csv_str)


def get_df_from_url(url=None):
    if not url:
        url = "https://raw.githubusercontent.com/atoye1/colab_notebook/main/btc.csv"
    df = pd.read_csv(url)
    return df


def get_daily_total(df):
    df_daily_total = df.groupby(['년월일', '역번호', '역명'])[
        '합계'].first().reset_index()
    df_daily_total['년월일'] = pd.to_datetime(df_daily_total['년월일'])
    return df_daily_total


def get_processed_df(df):
    # 필요없어진 합계 컬럼을 없앤다.
    df.drop(columns='합계', inplace=True)

    # 문자열인 년월일 컬럼을 datetime 자료형으로 변경한다.
    df['년월일'] = pd.to_datetime(df['년월일'])

    # 시간에 따라 id_vars를 녹인다. 결과적으로 로우가 value_vars 크기만큼 늘어난다.
    df_melted = df.melt(id_vars=['역번호', '역명', '년월일', '요일', '구분'], value_vars=['01시-02시', '02시-03시', '03시-04시',
                                                                              '04시-05시', '05시-06시', '06시-07시', '07시-08시', '08시-09시', '09시-10시',
                                                                              '10시-11시', '11시-12시', '12시-13시', '13시-14시', '14시-15시', '15시-16시',
                                                                              '16시-17시', '17시-18시', '18시-19시', '19시-20시', '20시-21시', '21시-22시',
                                                                              '22시-23시', '23시-24시', '24시-01시'])
    # melt가 적용된 결과를 다시 정리한다. datetime객체로 바꾸고, 필요없는 컬럼을 드롭한다.
    df_melted['hour'] = df_melted['variable'].str.slice(0, 2).astype(int)
    df_melted['timestamp'] = pd.to_datetime(
        df_melted['년월일']) + pd.to_timedelta(df_melted['hour'], unit='h')
    df_melted.drop(columns=['년월일', 'variable', 'hour'], inplace=True)
    df_melted.rename(columns={'value': 'passenger_count'}, inplace=True)

    # Sort the dataframe by timestamp
    df_melted.sort_values('timestamp', inplace=True)
    # 인덱스를 재정리해서 마지막 df형식으로 바꾼다.
    df_melted.reset_index(drop=True, inplace=True)
    df_final = df_melted.set_index(['역번호', 'timestamp'])
    return df_final


raw_df = get_df_from_url()
daily_total = get_daily_total(raw_df)
final_df = get_processed_df(raw_df)
print(final_df)

                              역명 요일  구분 passenger_count
역번호 timestamp                                          
95  2023-01-01 01:00:00  다대포해수욕장  일  승차               0
224 2023-01-01 01:00:00       냉정  일  하차               7
106 2023-01-01 01:00:00       대티  일  하차               5
    2023-01-01 01:00:00       대티  일  승차               4
225 2023-01-01 01:00:00       주례  일  승차               2
...                          ... ..  ..             ...
231 2023-05-01 00:00:00     구  남  일  승차               5
230 2023-05-01 00:00:00     모  라  일  하차              20
    2023-05-01 00:00:00     모  라  일  승차               0
235 2023-05-01 00:00:00     화  명  일  하차              84
414 2023-05-01 00:00:00     안  평  일  하차              18

[645120 rows x 4 columns]


In [3]:
# 역명을 통일한다.
final_df['역명_fixed'] = [''.join(i.split()) for i in final_df['역명']]
# 통행량을 숫자형 자료로 변경한다.
temp_list = []
for i in final_df['passenger_count']:
  # print(type(i))
  try:
    i = i.replace(',','')
    i = int(i)
    # print('replaced')
  except:
    pass
  temp_list.append(i)
final_df['passenger_count_int'] = temp_list

In [4]:
final_df.columns

Index(['역명', '요일', '구분', 'passenger_count', '역명_fixed', 'passenger_count_int'], dtype='object')

In [5]:
df_summed = final_df.groupby(['역번호', 'timestamp', '역명', '요일', '역명_fixed']).agg({'passenger_count_int': 'sum'}).reset_index()
df_summed.head(10)

,역번호,timestamp,역명,요일,역명_fixed,passenger_count_int
0,95,2023-01-01 01:00:00,다대포해수욕장,일,다대포해수욕장,2
1,95,2023-01-01 02:00:00,다대포해수욕장,일,다대포해수욕장,0
2,95,2023-01-01 03:00:00,다대포해수욕장,일,다대포해수욕장,0
3,95,2023-01-01 04:00:00,다대포해수욕장,일,다대포해수욕장,9
4,95,2023-01-01 05:00:00,다대포해수욕장,일,다대포해수욕장,79
5,95,2023-01-01 06:00:00,다대포해수욕장,일,다대포해수욕장,1407
6,95,2023-01-01 07:00:00,다대포해수욕장,일,다대포해수욕장,2496
7,95,2023-01-01 08:00:00,다대포해수욕장,일,다대포해수욕장,1831
8,95,2023-01-01 09:00:00,다대포해수욕장,일,다대포해수욕장,505
9,95,2023-01-01 10:00:00,다대포해수욕장,일,다대포해수욕장,400


In [6]:
final_df = final_df.reset_index()

In [8]:
fig = go.Figure()
# for station in final_df['역명'].unique():
# 아래에 있는 역들을 대상으로 시각화 한 결과
for station in ['1서면','2서면','센텀시티','하단','부산역','1연산','3연산','2덕천','3덕천','자갈치','부산대','경성대부경대', '1동래']:
    df_station = df_summed[df_summed['역명_fixed'] == station]
    fig.add_trace(go.Scatter(x=df_station['timestamp'], y=df_station['passenger_count_int'], name=station,
                 # 커스텀 호버 템플릿으로 호버링시 보이는 값을 지정할 수 있다.
                  hovertemplate =
                             '<b>%{y}</b><br><br>' +
                             '역명: %{text}<br>' +
                             '요일: %{customdata}',
                             text=df_station['역명'],
                             customdata=df_station['요일']))

# Add range slider
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1h", step="hour", stepmode="backward"),
                dict(count=1, label="1d", step="day", stepmode="backward"),
                dict(count=7, label="1w", step="day", stepmode="backward"),
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
            ])
        ),
        rangeslider=dict(visible=True),
        type="date"
    )
)

fig.show()